In [ ]:
import json
import requests
from bs4 import BeautifulSoup as bts
import bs4
import pandas as pd
import requests
import time
import numpy as np
import pymongo
import certifi

In [ ]:
def getAndParseURL(url):
    result = requests.get(url,headers={"User-Agent":"Mozilla/5.0"})
    soup = bts(result.text, 'html.parser')
    return soup

In [ ]:
def harvest_recipes(recipes, result):
    for recipelink in recipes:
        html = getAndParseURL(recipelink)
               
        #FOOD NAMES
        food = html.find('div', class_= 'headline-wrapper')
        if food is None:
            continue
        
        food_name = food.text.strip()
        
        
        #COUNT of RATINGS
        rate = html.find('span', class_ = 'ugc-ratings-item elementFont__details')
        if rate is None:
            continue
        
        rating = rate.text.strip()
            
        
        #COUNT of REVIEWS
        review = html.find('a', class_ = 'ugc-ratings-link elementFont__detailsLink--underlined ugc-reviews-link')
        if review is None:
            review_count = np.NaN
        else:
            review_count = review.text.strip()
            
        
        #COUNT of PHOTOS
        photo = html.find('a', class_ = 'ugc-ratings-link elementFont__detailsLink--underlined ugc-photos-link')
        if photo is None:
            photo_count = np.NaN
        else:
            photo_count = photo.text.strip()
            
        
        #TOTAL TIME of PREPARING and COOKING
        times = html.findAll('div', class_ = 'recipe-meta-item-body elementFont__subtitle')        
        if len(times) == 5:
            total_time = times[1].text.strip()
        elif len(times) == 6:
            total_time = times[2].text.strip()
        elif len(times) < 5:
            total_time = np.NaN       
            

        #INGREDIENTS
        metadata_ing = html.find('section', 
                                 class_ = 'component recipe-ingredients recipeIngredients container interactive')
        if metadata_ing is None:
            continue
            
        ingredients = []
        for i in metadata_ing.findAll('span', class_ = "ingredients-item-name elementFont__body"):
            ingredients.append(i.text.strip())


        # RECIPES
        metadata_recipe = html.find('fieldset', class_ = 'instructions-section__fieldset')
        if metadata_recipe is None:
            continue

        recipe = []
        for i in metadata_recipe.findAll('div', class_ = "paragraph"):
            recipe.append(i.text.strip())


        # NUTRITIONS    
        metadata_nutr = html.find('div', class_ = 'recipeNutritionSectionBlock')
        if metadata_nutr is None:
            continue
            
        nutrition = metadata_nutr.find('div', class_ = "section-body").text.strip()

        #USERS,REVIEWS AND STARS ALTOGETHER
        chain = []

        metadata_review = html.findAll('div', class_ = 'feedback__reviewContent')
        if len(metadata_review) == 0:
            continue
        
        for i in metadata_review:
            if len(i.text) == 0:
                continue
            chain.append(i.text.strip())


        result.append([food_name, rating, review_count, photo_count, total_time, 
                       ingredients, recipe, nutrition, chain])
        time.sleep(2)

In [ ]:
def load_to_mongo(data, collection_name): # collection_name is string
    
    client = pymongo.MongoClient('mongodb+srv://<your username>:<your password>@cluster0.33bbx.mongodb.net/myFirstDatabase?retryWrites=true&w=majority', ssl_ca_certs=certifi.where())
    db = client.food_metadata
    collection = db[collection_name]
    
    data.reset_index(inplace=True)
    data_dict = data.to_dict("records")

    collection.insert_many(data_dict)

In [ ]:
result = []
recipes = []
for i in range(10, 15):
    start = i+1
    html = getAndParseURL("https://www.allrecipes.com/recipes/737/healthy-recipes/low-cholesterol/?page=% s" % start)
    for sonuc in html.findAll('div', class_= 'tout__contentHeadline'):  
        if ('article' in sonuc.a['href']) & ('gallery' in sonuc.a['href']):
            continue
            
        recipes.append("https://www.allrecipes.com/"+ sonuc.a['href'])
    harvest_recipes(recipes, result)   
    

column = ["food_name", "rating", "review_count", "photo_count", "total_time", 
          "ingredients", "recipe", "nutrition", "chain"]

dfMatch_unique = pd.DataFrame.from_records(result, columns=column)

load_to_mongo(dfMatch_unique, 'lowcholesterol')